# Quantum Espresso Automation
Run this notebook, to run DFT calculations on a compound, and graph the results

### Species
Compound in the format X2YZ
Structure either L21 or XA

In [13]:
import subprocess
import sys
from pathlib import Path
import os
import shutil
import pandas as pd
import re

X='Co'
Y='Ti'
Z='Sn'
compound="Co2TiSn"
structure="L21"
latticeEstimate = 10.56


absPath = os.path.abspath(os.path.dirname(__vsc_ipynb_file__))
structPath= os.path.join(absPath, "Results", compound, structure)
homeTree = os.path.join(absPath, "CubicHeuslerScripts", structure)
lookupPath = os.path.join(absPath, "LookupTable.csv")

if os.path.exists(structPath):
    shutil.rmtree(structPath)
shutil.copytree(homeTree, structPath)
os.chdir(structPath)

lookupTable = pd.read_csv(lookupPath, header=None, index_col=0, squeeze=True).to_dict()

/tmp/ipykernel_4103/3737638184.py:27: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  lookupTable = pd.read_csv(lookupPath, header=None, index_col=0, squeeze=True).to_dict()


### Ecut Convergence Testing

Energy cutoff calculations:
In this section, replace the values in ecutList as needed

In [15]:
ecutList = '50 60 70 80 90 100 120 140'
ecutBash = os.path.join(structPath, "FH_ecut_optzm.bash")
with open(ecutBash, 'r', encoding='utf-8') as file:
    data = file.readlines()

data[10] = "for ecut in "+ecutList+ "\n"
data[16] = re.sub("X2YZ", compound, data[16], count=0, flags=0)
data[26] = re.sub("[0-9]*\.[0-9]+", str(latticeEstimate), data[26], count=0, flags=0)
data[53] = lookupTable[X] + "\n"
data[54] = lookupTable[Y] + "\n"
data[55] = lookupTable[Z] + "\n"
data[58] = X+"     0.000000000         0.000000000         0.000000000" + "\n"
data[59] = X+"     0.500000000         0.500000000         0.500000000" + "\n"
data[60] = Y+"     0.250000000         0.250000000         0.250000000" + "\n"
data[61] = Z+"     0.750000000         0.750000000         0.750000000" + "\n"

with open('FH_ecut_optzm.bash', 'w', encoding='utf-8') as file:
    file.writelines(data)
output = []
ecutEstimate = subprocess.run([os.path.join(structPath, "FH_ecut_optzm.bash")], stdout=subprocess.PIPE, text=True)
for line in iter(ecutEstimate.stdout.readline, b''):
    output.append(float(line))
ecutEstimate.stdout.close()

/home/vagrant/QuantumEspressoScripts/Results/Co2TiSn/L21/FH_ecut_optzm.bash: line 85: warning: here-document at line 13 delimited by end-of-file (wanted `EOF')
/home/vagrant/QuantumEspressoScripts/Results/Co2TiSn/L21/FH_ecut_optzm.bash: line 86: syntax error: unexpected end of file


AttributeError: 'str' object has no attribute 'readline'

### Convergence Graphing

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParamsDefault
import numpy as np
%matplotlib inline
plt.rcParams["figure.dpi"]=150
plt.rcParams["figure.facecolor"]="white"

x, y = np.loadtxt('../src/silicon/etot-vs-ecutwfc.dat', delimiter=' ', unpack=True)
plt.plot(x, y, "o-", markersize=5, label='Etot vs ecutwfc')
plt.xlabel('ecutwfc (Ry)')
plt.ylabel('Etot (Ry)')
plt.legend(frameon=False)
plt.show()

### Kpoint Convergence Testing
In this section, replace the values in kpointList as needed


In [ ]:
kpointList = '6 8 10 12 14 16 18 20'

with open('FH_kpoint_optzm.bash', 'r', encoding='utf-8') as file:
    data = file.readlines()

data[11] = "for k in "+kpointList 
data[17] = "prefix        = '"+compound+"',"
data[27] = "celldm(1)   = "+latticeEstimate+"," 
data[54] = lookupTable[X]
data[55] = lookupTable[Y]
data[56] = lookupTable[Z]
data[59] = X+"     0.000000000         0.000000000         0.000000000"
data[60] = X+"     0.500000000         0.500000000         0.500000000"
data[61] = Y+"     0.250000000         0.250000000         0.250000000"
data[62] = Z+"     0.750000000         0.750000000         0.750000000"

with open('FH_kpoint_optzm.bash', 'w', encoding='utf-8') as file:
    file.writelines(data)

output = []
kpointEstimate = subprocess.run(["FH_kpoint_optzm.bash"], stdout=subprocess.PIPE, text=True)
for line in iter(kpointEstimate.stdout.readline, b''):
    output.append(float(line))
kpointEstimate.stdout.close()

### Kpoint Convergence Graphing

In [ ]:
x, y = np.loadtxt('../src/silicon/etot-vs-kpoint.dat', delimiter=' ', unpack=True)
plt.plot(x, y, "o-", markersize=5, label='Etot vs kpoints')
plt.xlabel('# kpoints')
plt.ylabel('Etot (Ry)')
plt.legend(frameon=False)
plt.show()

### Lattice Optimization
Enter lattice estimate here below, for parabola triangulation


In [ ]:
def latticeParabola(x1, x2, x3, y1, y2, y3, xP, yP):
    denom = (x1 - x2)(x1 - x3)(x2 - x3)
    a = (x3 * (y2 - y1) + x2 * (y1 - y3) + x1 * (y3 - y2))
    b = (x3^2 * (y1 - y2) + x2^2 * (y3 - y1) + x1^2 * (y2 - y3))
    c = (x2 * x3 * (x2 - x3) * y1 + x3 * x1 * (x3 - x1) * y2 + x1 * x2 * (x1 - x2) * y3)
    xP = -b/ (2 * a)
    yP = (c - b^2 / (4 * a)) / denom

inputParam = str(latticeEstimate) + " " + str(latticeEstimate + 0.001) + " " + str(latticeEstimate + 0.002)
output = []

latticeEstimate = subprocess.run(["FH_lattice_optzm.bash", inputParam], stdout=subprocess.PIPE, text=True)
for line in iter(latticeEstimate.stdout.readline, b''):
    output.append(float(line))
latticeEstimate.stdout.close()

xP = 0
yP = 0
latticeParabola(latticeEstimate, latticeEstimate+ 0.001, latticeEstimate+0.002, output[0], output[1], output[2],xP, yP)
print("lattice vertex x=",xP," and y=", yP)

### Lattice Optimization Convergence Graphing

In [ ]:
x, y = np.loadtxt('../src/silicon/etot-vs-alat.dat', delimiter=' ', unpack=True)
plt.plot(x, y, "o-", markersize=5, label='Etot vs alat')
plt.xlabel('alat (Bohr)')
plt.ylabel('Etot (Ry)')
plt.legend(frameon=False)
plt.show()

### Relaxation Run

In [ ]:
atomicPos = []
latticeVertex = xP
relaxationRun = subprocess.run(["FH_relax.bash", latticeVertex], stdout=subprocess.PIPE, text=True)
for line in iter(relaxationRun.stdout.readline, b''):
    if(count >= 0):
        atomicPos.append(line.split(None))
        count += 1
        if(count == 3):
            count = -1
    
    if(line.split(None)[0] == "ATOMIC_POSITION"):
        count = 0
relaxationRun.stdout.close()


### SCF Preparation

In [ ]:
with open('FH_scf.bash', 'r', encoding='utf-8') as file:
    data = file.readlines()

data[11] = "for k in "+latticeVertex
data[17] = "prefix        = '"+compound+"',"
data[27] = "celldm(1)   = "+latticeVertex+"," 
data[38] = "  occupations = 'smearing',"
data[54] = lookupTable[X]
data[55] = lookupTable[Y]
data[56] = lookupTable[Z]
data[59] = X+"     "+atomicPos[0][1]+"         "+atomicPos[0][2]+"         "+atomicPos[0][3]
data[60] = X+"     "+atomicPos[1][1]+"         "+atomicPos[1][2]+"         "+atomicPos[1][3]
data[61] = Y+"     "+atomicPos[2][1]+"         "+atomicPos[2][2]+"         "+atomicPos[2][3]
data[62] = Z+"     "+atomicPos[3][1]+"         "+atomicPos[3][2]+"         "+atomicPos[3][3]

with open('FH_kpoint_optzm.bash', 'w', encoding='utf-8') as file:
    file.writelines(data)

### SCF Run

In [ ]:
output = []
latticeVertex = xP
scfRun = subprocess.run(["FH_relax.bash", latticeVertex], stdout=subprocess.PIPE, text=True)
for line in iter(scfRun.stdout.readline, b''):
    if(line.split(None)[0] == "ATOMIC_POSITION"):
        x = [X]
        x.append(line.split(None))
        output.append(x)
    output.append(line)
scfRun.stdout.close()

print(output)

### DOS

In [ ]:
output = []

relxationRun = subprocess.run(["FH_relax.bash", xP], stdout=subprocess.PIPE, text=True)
for line in iter(relxationRun.stdout.readline, b''):
    output.append(float(line))
relaxationRun.stdout.close()

### DOS Graph

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParamsDefault
import numpy as np
%matplotlib inline

# load data
energy, dos, idos = np.loadtxt('../src/silicon/si_dos.dat', unpack=True)

# make plot
plt.figure(figsize = (12, 6))
plt.plot(energy, dos, linewidth=0.75, color='red')
plt.yticks([])
plt.xlabel('Energy (eV)')
plt.ylabel('DOS')
plt.axvline(x=6.642, linewidth=0.5, color='k', linestyle=(0, (8, 10)))
plt.xlim(-6, 16)
plt.ylim(0, )
plt.fill_between(energy, 0, dos, where=(energy < 6.642), facecolor='red', alpha=0.25)
plt.text(6, 1.7, 'Fermi energy', fontsize= med, rotation=90)
plt.show()

### Bands

In [ ]:
output = []

relxationRun = subprocess.run(["FH_relax.bash", xP], stdout=subprocess.PIPE, text=True)
for line in iter(relxationRun.stdout.readline, b''):
    output.append(float(line))
relaxationRun.stdout.close()

### Bands Graph

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParamsDefault
import numpy as np
%matplotlib inline

plt.rcParams["figure.dpi"]=150
plt.rcParams["figure.facecolor"]="white"
plt.rcParams["figure.figsize"]=(8, 6)

# load data
data = np.loadtxt('../src/silicon/si_bands.dat.gnu')

k = np.unique(data[:, 0])
bands = np.reshape(data[:, 1], (-1, len(k)))

for band in range(len(bands)):
    plt.plot(k, bands[band, :], linewidth=1, alpha=0.5, color='k')
plt.xlim(min(k), max(k))

# Fermi energy
plt.axhline(6.6416, linestyle=(0, (5, 5)), linewidth=0.75, color='k', alpha=0.5)
# High symmetry k-points (check bands_pp.out)
plt.axvline(0.8660, linewidth=0.75, color='k', alpha=0.5)
plt.axvline(1.8660, linewidth=0.75, color='k', alpha=0.5)
plt.axvline(2.2196, linewidth=0.75, color='k', alpha=0.5)
# text labels
plt.xticks(ticks= [0, 0.8660, 1.8660, 2.2196, 3.2802], \
           labels=['L', '$\Gamma$', 'X', 'U', '$\Gamma$'])
plt.ylabel("Energy (eV)")
plt.text(2.3, 5.6, 'Fermi energy', fontsize= small)
plt.show()